In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import joblib
import os
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt

2025-04-03 11:10:06.827198: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-03 11:10:06.914262: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743678606.949355   23405 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743678606.959469   23405 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743678607.031614   23405 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
train = pd.read_csv("../DATA/train.csv")

/tmp/ipykernel_23405/2678278497.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("../DATA/train.csv")


In [4]:
train = train.dropna()

In [5]:
train['Protocol'] = pd.to_numeric(train['Protocol'], errors='coerce') 
train = train.dropna(subset=['Protocol']) 
train['Protocol'] = train['Protocol'].astype(int)

In [6]:
columns = list(train.columns) 
columns = [col for col in columns if col not in ['Fwd PSH Flags', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'Protocol', 'Label']]

In [7]:
x = train.drop(columns=['Label'])
y = train['Label']

In [9]:
x[columns] = x[columns].apply(pd.to_numeric, errors='coerce')
f=['Fwd PSH Flags', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt']
x[f] = x[f].apply(pd.to_numeric, errors='coerce')
x = x.fillna(0)

In [10]:
sc = joblib.load('../FEATURE_EXTRACTION/scaler.pkl')  
x[columns] = sc.transform(x[columns])

In [11]:
le = LabelEncoder()
ye = le.fit_transform(y)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(x, ye, test_size=0.15, random_state=42, stratify=ye)

In [13]:
class_counts = np.bincount(y_train)
total_samples = np.sum(class_counts)
class_weights = {i: total_samples / (len(class_counts) * count) for i, count in enumerate(class_counts)}

In [14]:
dbn_weights = [
                np.load("../FEATURE_EXTRACTION/rbm_layer_1_weights.npz"),
                np.load("../FEATURE_EXTRACTION/rbm_layer_2_weights.npz"),
                np.load("../FEATURE_EXTRACTION/rbm_layer_3_weights.npz")
            ]

In [18]:
def build_high_accuracy_mlp(input_dim, num_classes, dbn_weights=None):
    """Build MLP model with optional DBN weight initialization"""
    inputs = Input(shape=(input_dim,))
    
    # Layer 1: Initial layer with DBN weights if available
    x = Dense(512, activation='swish',
                 kernel_initializer=tf.constant_initializer(dbn_weights[0]['W']),
                 bias_initializer=tf.constant_initializer(dbn_weights[0]['bh']),
                 kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(inputs)
    
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    # Layer 2
    x = Dense(256, activation='swish', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    
    # Layer 3
    x = Dense(128, activation='swish', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    # Layer 4
    x = Dense(64, activation='swish', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    # Output layer
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
model = build_high_accuracy_mlp(X_train.shape[1], len(np.unique(y)), dbn_weights)


TypeError: Eager execution of tf.constant with unsupported shape. Tensor [[-4.9986500e-02 -3.0140501e-02 -1.6280431e-01 ... -4.0832300e-02
  -9.9839736e-03  4.7602769e-02]
 [ 4.6088340e+04  4.6085285e+04  4.6086820e+04 ...  4.6087000e+04
   4.6088379e+04  4.6088492e+04]
 [-7.9239684e-01 -7.2697359e-01 -7.2565013e-01 ... -1.1235443e+00
  -7.2513372e-01 -6.6785139e-01]
 ...
 [ 1.8563789e-01  2.0050332e-02  1.4230958e-01 ... -1.0456318e-01
   5.8039881e-02  1.4489168e-01]
 [ 1.7153993e-01 -8.6636402e-02  4.2609505e-02 ...  1.8159348e-01
   5.4826528e-02 -1.3454488e-01]
 [-2.0515776e+00 -1.9022062e+00 -2.1558654e+00 ... -1.9476743e+00
  -2.0807083e+00 -2.1908538e+00]] (converted from [[-4.9986500e-02 -3.0140501e-02 -1.6280431e-01 ... -4.0832300e-02
  -9.9839736e-03  4.7602769e-02]
 [ 4.6088340e+04  4.6085285e+04  4.6086820e+04 ...  4.6087000e+04
   4.6088379e+04  4.6088492e+04]
 [-7.9239684e-01 -7.2697359e-01 -7.2565013e-01 ... -1.1235443e+00
  -7.2513372e-01 -6.6785139e-01]
 ...
 [ 1.8563

: 

In [ ]:
model

In [ ]:
def train_with_boosted_accuracy(model, X_train, y_train, X_val, y_val, class_weights):
    """4-phase training strategy for optimal accuracy"""
    callbacks = [
        EarlyStopping(monitor='val_accuracy', patience=25, restore_best_weights=True, mode='max', verbose=1),
        ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=10, min_lr=1e-7, mode='max', verbose=1),
        ModelCheckpoint('best_mlp_model.keras', monitor='val_accuracy', save_best_only=True, mode='max'),
        TensorBoard(log_dir='./logs', histogram_freq=1)
    ]
    
    print("\n=== Starting 4-Phase Training ===")
    
    # Phase 1: Warmup (train only top layers)
    for layer in model.layers[:-6]:
        layer.trainable = False
    
    model.compile(optimizer=Adam(learning_rate=1e-3),
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    
    print("\nPhase 1: Warmup Training (Top Layers Only)")
    history1 = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=50,
        batch_size=2048,
        class_weight=class_weights,
        callbacks=callbacks,
        verbose=1
    )
    
    # Phase 2: Intermediate (unfreeze middle layers)
    for layer in model.layers[:-10]:
        layer.trainable = False
    
    model.compile(optimizer=Adam(learning_rate=5e-4),
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    
    print("\nPhase 2: Intermediate Training (Middle Layers)")
    history2 = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=60,
        batch_size=1024,
        class_weight=class_weights,
        callbacks=callbacks,
        verbose=1
    )
    
    # Phase 3: Full Training (all layers)
    for layer in model.layers:
        layer.trainable = True
    
    model.compile(optimizer=Nadam(learning_rate=1e-4),
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    
    print("\nPhase 3: Full Network Training")
    history3 = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=80,
        batch_size=512,
        class_weight=class_weights,
        callbacks=callbacks,
        verbose=1
    )
    
    # Phase 4: Fine-tuning (very low LR)
    model.compile(optimizer=Nadam(learning_rate=1e-5),
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    
    print("\nPhase 4: Fine-Tuning")
    history4 = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=100,
        batch_size=256,
        class_weight=class_weights,
        callbacks=callbacks,
        verbose=1
    )
    
    return model, (history1, history2, history3, history4)

In [ ]:
 model, histories = train_with_boosted_accuracy(
            model, X_train, y_train, X_test, y_test, class_weights
        )